In [376]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
from collections import Counter
import time
import re
import jieba
from tqdm import tqdm 

In [347]:
df_all = pd.read_csv('./data/20221007_20231006_all.csv')

In [395]:
df_mapping = pd.read_csv('./data/sa340.csv')
short_names = list(set(df_mapping['city_shortname']))
print(len(short_names))

with open ('./data/family_name.txt') as f:
    family_name = f.read()
family_names = family_name.split(',')
family_names = family_names[0:100]
print(len(family_names))

340
100


In [313]:
# 字典排序
def take_degree(tup):
    return tup[1]

def get_result_sort(dict_data):
    item_arr = []
    for i in dict_data:
        item_arr.append((i, dict_data.get(i)))
    item_arr.sort(key=take_degree, reverse=True)
    
    res = []
    rank = 0
    for i in item_arr:
        rank += 1
        res.append([i[0], i[1], rank])
    return res

In [368]:
def build_stopwords(city_name):
    data = []
    # common sense
    data.append(city_name + '路')
    data.append(city_name + '东路')
    data.append(city_name + '西路')
    data.append(city_name + '南路')
    data.append(city_name + '北路')
    data.append(city_name + '中路')
    data.append(city_name + '街')
    data.append(city_name + '镇')
    data.append(city_name + '县')
    data.append(city_name + '区')
    data.append(city_name + '村')

    
    data.append(city_name + '公园')
    data.append(city_name + '先生')
    data.append(city_name + '大道')
    data.append(city_name + '一路')
    data.append(city_name + '二路')
    data.append(city_name + '三路')
    data.append(city_name + '四路')
    data.append(city_name + '五路')
    data.append(city_name + '六路')
    data.append(city_name + '七路')
    data.append(city_name + '八路')
    data.append(city_name + '九路')
    data.append(city_name + '十路')
    data.append(city_name + '环路')
    data.append(city_name + '的路')
    data.append(city_name + '东一路')
    data.append(city_name + '西一路')
    data.append(city_name + '南一路')
    data.append(city_name + '北一路')
    data.append(city_name + '东二路')
    data.append(city_name + '西二路')
    data.append(city_name + '南二路')
    data.append(city_name + '北二路')
    data.append(city_name + '东街')
    data.append(city_name + '西街')
    data.append(city_name + '南街')
    data.append(city_name + '北街')
    data.append(city_name + '支路')
    data.append(city_name + '桥')
    data.append(city_name + '外国语学校')
    
    # 过滤底部无关新闻的超链接
    data.append('@' + city_name)
    data.append(city_name + '广场')
    data.append(city_name + '池')
    
    # special common sense
    data.append('吉林省')
    data.append('北京时间')
    data.append('入海口')
    data.append('海口岸')
    data.append('延安精神')
    data.append('重大理论')
    data.append('亚运城')
    data.append('日照时长')
    data.append('东西部')
    
    data.append('南海东') 
    data.append('海东西') 
    data.append('海东南')
    data.append('海东北')
    data.append('海南部')
    
    data.append('南海西')
    data.append('海西部')
    data.append('海西北')
    data.append('海西南')
    data.append('海西区')
    data.append('海西湖')
    
    data.append('海北部')
    data.append('渤海北')
    data.append('南海北')
    data.append('东海北')
    data.append('海口岸')
    data.append('出海口')
    
     # extra knowledge
    data.append('三明治')
    data.append('端午安康')
    data.append('孙中山')
    data.append('适中')
    data.append('重庆小面')
    data.append('重庆麻辣')
    data.append('重庆面庄')
    data.append('隆重庆祝')
    data.append('长春百子图')
    data.append('春分')
    data.append('兰州拉面')
    data.append('兰州牛肉面')
    data.append('兰州牛肉拉面')
    data.append('兰州传统牛肉面')
    data.append('澳大利亚昆士兰州')
    data.append('马里兰州')
    data.append('宁德时代')
    data.append('山地') 
    
    # add hundred family names
    for f in family_names:
        data.append(f + city_name)
        
        
    return data

In [315]:
"""
Replace the words by using stopwords 
"""
def replace_citys_stop_words(text, stopwords):
    # Some sensitive words need to be removed here
    if 'xxx' in text:
        return ''
    
    # 因为编辑的外链习惯是不同的
    
    if '来源' in text:
        text = text.split('来源')[0]
        
    if '编辑' in text:
        text = text.split('编辑')[0]
        
    if '往期阅读' in text:
        text = text.split('往期阅读')[0]
        
        
    if '推荐阅读' in text:
        text = text.split('推荐阅读')[0]
    
        
    for i in stopwords:
        text = text.replace(i, '')
    return text

"""
城市简称转全称
"""
def name2shortname(name):
    return list(set(df_mapping[df_mapping['name'] == name]['city_shortname']))[0]

In [316]:
# Statistical citation
def tj(city_name):
    target_city = city_name
    shortname = name2shortname(target_city) 
    sns = shortname.split('+')
    # Using abbreviations is to generate a vocabulary
    stopwords = build_stopwords(sns[0])
    cities_list = list(df_mapping['name'])
      
    datas = []
    for i in cities_list:
        # like a paper citation
        #///////////////self///////////////////
        # If it is itself, it is directly treated as 0
        if i == city_name:
            datas.append((i, 0))
            continue
            
        #///////////////others///////////////////
        # The city dealt with them one by one
        origin_city = i
        city_df = df_all[df_all.city_name==origin_city]
        times = 0
        
        for i, row in city_df.iterrows():
            # text of one article 
            text = row['text']
            
            for name in sns:
                if name in text:
                    # 1.replace
                    text = replace_citys_stop_words(text, stopwords)
                    # 2.segmentation
                    cut_ts = jieba.lcut(text)
                    if name in cut_ts:
                        times += 1
                        break
        datas.append((origin_city, times))
    return datas


# key： 例如中山
# origin_city：重庆市
# debug_in('沈阳', '西安市')
def debug_in(key, origin_city):
    c = 0
    texts = list(df_all[df_all.city_name==origin_city]['text'])
    links = list(df_all[df_all.city_name==origin_city]['link'])
    
    for i in range(len(texts)):
        stopwords = build_stopwords(key)
        text = texts[i]
        
        if key in text:
            # 1.替换
            text = replace_citys_stop_words(texts[i], stopwords)
            # 2.分词
            cut_ts = jieba.lcut(text)
            if key in cut_ts:
                print(texts[i])
                print(links[i])
                c += 1
    print(c)

In [396]:
city_name_do = '普洱市'
df_mapping[df_mapping.name==city_name_do]

,citycode,adcode,name,level,center,pro_name,pro_adcode,gzh,city_shortname,pro_shortname,en_name
265,879,530800,普洱市,city,"100.966011,22.825229",云南省,530000,普洱发布,普洱,云南,Pu'er


In [397]:
data = tj(city_name_do)

In [398]:
result_dict = {key: value for key, value in data}
get_result_sort(result_dict)

[['西双版纳傣族自治州', 234, 1],
 ['文山壮族苗族自治州', 157, 2],
 ['玉溪市', 128, 3],
 ['红河哈尼族彝族自治州', 126, 4],
 ['大理白族自治州', 102, 5],
 ['楚雄彝族自治州', 70, 6],
 ['保山市', 69, 7],
 ['昆明市', 64, 8],
 ['德宏傣族景颇族自治州', 58, 9],
 ['曲靖市', 49, 10],
 ['昭通市', 42, 11],
 ['临沧市', 21, 12],
 ['怒江傈僳族自治州', 15, 13],
 ['娄底市', 11, 14],
 ['迪庆藏族自治州', 9, 15],
 ['七台河市', 7, 16],
 ['苏州市', 7, 17],
 ['陇南市', 7, 18],
 ['丹东市', 6, 19],
 ['抚州市', 6, 20],
 ['上海市', 6, 21],
 ['齐齐哈尔市', 5, 22],
 ['宜春市', 5, 23],
 ['九江市', 5, 24],
 ['南充市', 5, 25],
 ['拉萨市', 5, 26],
 ['双鸭山市', 4, 27],
 ['中山市', 4, 28],
 ['固原市', 4, 29],
 ['重庆市', 4, 30],
 ['承德市', 3, 31],
 ['呼伦贝尔市', 3, 32],
 ['镇江市', 3, 33],
 ['常州市', 3, 34],
 ['铜陵市', 3, 35],
 ['宣城市', 3, 36],
 ['上饶市', 3, 37],
 ['潍坊市', 3, 38],
 ['德州市', 3, 39],
 ['开封市', 3, 40],
 ['荆州市', 3, 41],
 ['咸宁市', 3, 42],
 ['阳江市', 3, 43],
 ['南宁市', 3, 44],
 ['三亚市', 3, 45],
 ['成都市', 3, 46],
 ['遵义市', 3, 47],
 ['天水市', 3, 48],
 ['玉树藏族自治州', 3, 49],
 ['吴忠市', 3, 50],
 ['台湾省', 3, 51],
 ['晋城市', 2, 52],
 ['包头市', 2, 53],
 ['哈尔滨市', 2, 54],
 ['金华市', 2, 55],
 

In [399]:
sum_ = 0
for i in data:
    sum_ += i[1]
    
sum_

1445

In [390]:
mx = pd.read_csv('./data/city_gzh_aja_martix6.csv')
np.sum(mx[city_name_do])

1445

In [383]:
source_cities = list(mx['city'])
sum_ = 0
ds = []
for i in source_cities:
    sum_ += result_dict.get(i)
    print(i, result_dict.get(i))
    ds.append(result_dict.get(i))
    
mx[city_name_do] = ds

龙岩市 0
齐齐哈尔市 6
黔西南布依族苗族自治州 0
黔南布依族苗族自治州 1
黔东南苗族侗族自治州 4
黑河市 0
黄石市 0
黄山市 2
黄南藏族自治州 0
黄冈市 0
鹰潭市 2
鹤岗市 0
鹤壁市 0
鸡西市 0
驻马店市 0
马鞍山市 0
香港特别行政区 0
韶关市 0
鞍山市 1
青岛市 15
雅安市 0
随州市 0
陇南市 6
阿里地区 0
阿拉善盟 0
阿坝藏族羌族自治州 1
阿勒泰地区 0
阿克苏地区 0
阳泉市 0
阳江市 0
防城港市 0
阜阳市 3
阜新市 0
长治市 0
长沙市 0
长春市 0
镇江市 8
锦州市 0
锡林郭勒盟 0
银川市 0
铜陵市 4
铜川市 0
铜仁市 0
铁岭市 0
钦州市 0
金昌市 3
金华市 10
重庆市 2
酒泉市 2
鄂州市 0
鄂尔多斯市 1
郴州市 1
郑州市 0
邵阳市 0
邯郸市 1
那曲市 2
邢台市 0
遵义市 3
遂宁市 0
通辽市 1
通化市 0
迪庆藏族自治州 0
连云港市 0
运城市 0
达州市 0
辽阳市 0
辽源市 0
赤峰市 0
赣州市 0
资阳市 0
贺州市 0
贵阳市 3
贵港市 1
许昌市 0
西安市 0
西宁市 0
西双版纳傣族自治州 0
襄阳市 0
衢州市 0
衡阳市 2
衡水市 0
蚌埠市 4
葫芦岛市 0
营口市 0
萍乡市 2
菏泽市 13
莆田市 0
荆门市 1
荆州市 0
茂名市 0
苏州市 35
芜湖市 3
舟山市 0
自贡市 0
肇庆市 0
聊城市 2
绵阳市 0
绥化市 9
绍兴市 0
红河哈尼族彝族自治州 0
秦皇岛市 0
福州市 0
石家庄市 0
石嘴山市 0
眉山市 1
盘锦市 0
盐城市 0
益阳市 0
百色市 1
白银市 0
白山市 1
白城市 1
甘孜藏族自治州 0
甘南藏族自治州 1
珠海市 0
玉溪市 1
玉树藏族自治州 1
玉林市 0
牡丹江市 0
焦作市 0
烟台市 1
濮阳市 0
澳门特别行政区 7
潮州市 0
潍坊市 41
漳州市 0
漯河市 1
滨州市 17
滁州市 1
湛江市 2
湘西土家族苗族自治州 0
湘潭市 0
湖州市 0
渭南市 1
温州市 0
清远市 0
深圳市 1
淮安市 6
淮南市 2
淮北市 0
淄博市 9
海西蒙古族藏族自治州 2
海口市 1
海南藏族自治州 0
海北藏族自治州 2
海东市 0
济宁市 4


In [384]:
mx.to_csv('./data/city_gzh_aja_martix6.csv', index=None)
np.sum(mx[city_name_do])

561